In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOllama
from langchain.chains import ConversationalRetrievalChain
import os


In [7]:
loader = PyPDFLoader("updated-tcs---rtp-international-stipend (1).pdf") # Loads the PDF
documents = loader.load() 


In [8]:
documents

[Document(metadata={'producer': 'Adobe PDF Library 24.3.212', 'creator': 'Acrobat PDFMaker 24 for Word', 'creationdate': '2024-10-18T10:03:32+11:00', 'author': 'Richard Prekodravac', 'comments': '', 'company': 'University of Sydney', 'keywords': '', 'moddate': '2024-10-18T10:03:35+11:00', 'sourcemodified': 'D:20241017230308', 'subject': '', 'title': '', 'source': 'updated-tcs---rtp-international-stipend (1).pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Research Training Program Stipend Scholarship \n(International) \n1 \nLast Updated: September 2024 \nSubject to any changes announced by the Commonwealth Government \nTerms and Conditions  \n \nEligibility \n \n1. Applicants must be an international student at the time of award.  \n \n2. Research Training Program (RTP) Stipend International Scholarship (‘the Scholarship’) holders \nare expected to enrol in a full-time postgraduate research degree.  A part-time RTP Stipend \nScholarship* may be approved where the Un

In [9]:
faiss_path = "faiss"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/g2/8r_t2nhj151b_tp52sl6mc8h0000gn/T/ipykernel_76660/2891069498.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/braintip/AIBOOKCLUB/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
if os.path.exists(faiss_path):
        db = FAISS.load_local(faiss_path, embeddings)
        print("✅ Loaded existing FAISS index.")
else:
    print("⚙️ Creating new FAISS index...")
    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(faiss_path)
    print("✅ FAISS index saved.")

⚙️ Creating new FAISS index...
✅ FAISS index saved.


In [ ]:
import ollama
model = "llama3.1"
def get_response(query,chat_history):
    data = db.similarity_search(query)
    page_content = [x.page_content for x in data]
    prompt =f"""
    Here is a conversation history of the chat.
    history : {chat_history}
you are a conversational agent that helps with user query from the data provided.
data provided : {page_content}
query : {query}
"""
    
    response = ollama.generate(prompt=prompt,model=model)
    return response.response

    

In [21]:
db.similarity_search("On what basis the scholarship will be provided", k= 2)

[Document(id='e237b43f-980f-4cf1-8fb8-a6ecfa280cd5', metadata={'producer': 'Adobe PDF Library 24.3.212', 'creator': 'Acrobat PDFMaker 24 for Word', 'creationdate': '2024-10-18T10:03:32+11:00', 'author': 'Richard Prekodravac', 'comments': '', 'company': 'University of Sydney', 'keywords': '', 'moddate': '2024-10-18T10:03:35+11:00', 'sourcemodified': 'D:20241017230308', 'subject': '', 'title': '', 'source': 'updated-tcs---rtp-international-stipend (1).pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Research Training Program Stipend Scholarship \n(International) \n1 \nLast Updated: September 2024 \nSubject to any changes announced by the Commonwealth Government \nTerms and Conditions  \n \nEligibility \n \n1. Applicants must be an international student at the time of award.  \n \n2. Research Training Program (RTP) Stipend International Scholarship (‘the Scholarship’) holders \nare expected to enrol in a full-time postgraduate research degree.  A part-time RTP Stipend 

In [23]:
query = "On what basis the scholarship will be provided"

In [24]:
data = db.similarity_search(query)
page_content = [x.page_content for x in data]

In [25]:
page_content

['Research Training Program Stipend Scholarship \n(International) \n1 \nLast Updated: September 2024 \nSubject to any changes announced by the Commonwealth Government \nTerms and Conditions  \n \nEligibility \n \n1. Applicants must be an international student at the time of award.  \n \n2. Research Training Program (RTP) Stipend International Scholarship (‘the Scholarship’) holders \nare expected to enrol in a full-time postgraduate research degree.  A part-time RTP Stipend \nScholarship* may be approved where the University is satisfied that there are special \ncircumstances beyond the student’s control (for example medical conditions, financial hardship, \ncarer’s responsibilities).  \n \n*A part-time scholarship may have tax implications and student should seek tax advice from a \nregistered tax agent. International students will only receive approval to study part-time if it is \npermitted under their visa.  \n \n3. Applicants must apply to be considered for RTP Scholarships at the

In [26]:
prompt =f"""
you are a conversational agent that helps with user query from the data provided.
data provided : {page_content}
query : {query}
"""


In [27]:
prompt

'\nyou are a conversational agent that helps with user query from the data provided.\ndata provided : [\'Research Training Program Stipend Scholarship \\n(International) \\n1 \\nLast Updated: September 2024 \\nSubject to any changes announced by the Commonwealth Government \\nTerms and Conditions  \\n \\nEligibility \\n \\n1. Applicants must be an international student at the time of award.  \\n \\n2. Research Training Program (RTP) Stipend International Scholarship (‘the Scholarship’) holders \\nare expected to enrol in a full-time postgraduate research degree.  A part-time RTP Stipend \\nScholarship* may be approved where the University is satisfied that there are special \\ncircumstances beyond the student’s control (for example medical conditions, financial hardship, \\ncarer’s responsibilities).  \\n \\n*A part-time scholarship may have tax implications and student should seek tax advice from a \\nregistered tax agent. International students will only receive approval to study par

In [31]:
import ollama
model = "llama3.1"


In [32]:
response = ollama.generate(prompt=prompt,model=model)

RemoteProtocolError: Server disconnected without sending a response.

In [14]:
splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = splitter.split_documents(documents)

In [15]:
docs

[Document(metadata={'producer': 'Adobe PDF Library 24.3.212', 'creator': 'Acrobat PDFMaker 24 for Word', 'creationdate': '2024-10-18T10:03:32+11:00', 'author': 'Richard Prekodravac', 'comments': '', 'company': 'University of Sydney', 'keywords': '', 'moddate': '2024-10-18T10:03:35+11:00', 'sourcemodified': 'D:20241017230308', 'subject': '', 'title': '', 'source': 'updated-tcs---rtp-international-stipend (1).pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Research Training Program Stipend Scholarship \n(International) \n1 \nLast Updated: September 2024 \nSubject to any changes announced by the Commonwealth Government \nTerms and Conditions  \n \nEligibility \n \n1. Applicants must be an international student at the time of award.  \n \n2. Research Training Program (RTP) Stipend International Scholarship (‘the Scholarship’) holders \nare expected to enrol in a full-time postgraduate research degree.  A part-time RTP Stipend \nScholarship* may be approved where the Un